In [9]:
import cv2
import numpy as np
import csv
import os

In [10]:
# Define color ranges in HSV
color_ranges = {
    'red': ([0, 120, 70], [10, 255, 255]),
    'green': ([36, 25, 25], [86, 255, 255]),
    'blue': ([94, 80, 2], [126, 255, 255]),
    'white': ([0, 0, 200], [180, 20, 255]),
    'yellow': ([15, 100, 100], [35, 255, 255])}

In [11]:
# Define quadrants
def define_quadrants(frame):
    height, width = frame.shape[:2]
    quadrants = {
        1: (0, height // 2, 0, width // 2),
        2: (0, height // 2, width // 2, width),
        3: (height // 2, height, 0, width // 2),
        4: (height // 2, height, width // 2, width)
    }
    return quadrants

In [12]:
# Function to write events to CSV
def write_events_to_csv(events, csv_filename):
    with open(csv_filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Time', 'Quadrant Number', 'Ball Colour', 'Type'])
        for event in events:
            writer.writerow([event['time'], event['quadrant'], event['color'], event['action']])

In [13]:
## Function to overlay text on the frame
def overlay_text(frame, text, position):
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(frame, text, position, font, 0.6, (255, 255, 255), 2, cv2.LINE_AA)


In [14]:
# Function to track balls and create processed video
def track_balls(video_path, output_video_path, output_csv_path):
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = 0
    events = []
    ball_positions = {}

    # Define video writer
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (int(cap.get(3)), int(cap.get(4))))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        quadrants = define_quadrants(frame)
        for color, (lower, upper) in color_ranges.items():
            lower = np.array(lower, dtype="uint8")
            upper = np.array(upper, dtype="uint8")
            mask = cv2.inRange(hsv, lower, upper)
            contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            for contour in contours:
                if cv2.contourArea(contour) < 500:  # Filter small contours
                    continue
                (x, y, w, h) = cv2.boundingRect(contour)
                cx, cy = x + w // 2, y + h // 2
                current_quadrant = None
                for q, (y1, y2, x1, x2) in quadrants.items():
                    if x1 <= cx <= x2 and y1 <= cy <= y2:
                        current_quadrant = q
                        break

                ball_id = (color, len(contour))
                if ball_id not in ball_positions:
                    ball_positions[ball_id] = {'last_seen': None}

                last_seen_quadrant = ball_positions[ball_id]['last_seen']
                timestamp = frame_count / fps
                if current_quadrant != last_seen_quadrant:
                    if last_seen_quadrant is not None:
                        events.append({
                            'time': timestamp,
                            'color': color,
                            'quadrant': last_seen_quadrant,
                            'action': 'exit'
                        })
                        overlay_text(frame, f"Exit {last_seen_quadrant}", (x, y - 10))
                    if current_quadrant is not None:
                        events.append({
                            'time': timestamp,
                            'color': color,
                            'quadrant': current_quadrant,
                            'action': 'enter'
                        })
                        overlay_text(frame, f"Entry {current_quadrant}", (x, y - 10))
                    ball_positions[ball_id]['last_seen'] = current_quadrant

                cv2.circle(frame, (cx, cy), 5, (255, 255, 255), -1)
                overlay_text(frame, color, (x, y + h + 20))

        out.write(frame)
        frame_count += 1

    cap.release()
    out.release()
    write_events_to_csv(events, output_csv_path)

In [15]:
# Paths to input and output files
input_video_path = 'AI Assignment video.mp4'
output_video_path = 'processed_video.avi'
output_csv_path = 'events.csv'

In [16]:
# Run the tracking function
track_balls(input_video_path, output_video_path, output_csv_path)

print(f"Processed video saved at {output_video_path}")
print(f"Event data saved at {output_csv_path}")


Processed video saved at processed_video.avi
Event data saved at events.csv
